In [1]:
import cobra
from scipy.io import savemat 
import os
import tqdm

# Load SBML Model

In [2]:
model=cobra.io.read_sbml_model('../../Model/iCH360red/iCH360red.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12


# Export condition-specific data structures to use with EFMTool in Matlab

In [3]:
def export_matlab_structure(model,
                            carbon_source,
                            condition,
                            carbon_source_specific_remove=None,
                            condition_specific_remove=None,
                            out_path='./matlab_models/'):

        with model as m:
            m.reactions.EX_glc__D_e.bounds=(0,1000)
            m.reactions.get_by_id(f'EX_{carbon_source}_e').lower_bound=-10
            if condition=='aerobic':
                m.reactions.get_by_id('EX_o2_e').lower_bound=-1000
            elif condition=='anaerobic':
                m.reactions.get_by_id('EX_o2_e').lower_bound=0

            else:
                raise ValueError
            print(f'Exporting EFMTools Matlab structure for {carbon_source} {condition}....')
            removed_ids=[]
            #Remove carbon source specific reactions
            if carbon_source_specific_remove is not None:
                for cs in carbon_source_specific_remove.keys():
                    if carbon_source==cs:
                        removed_rxns=[m.reactions.get_by_id(r) for r in carbon_source_specific_remove[cs] if r in model.reactions]
                        removed_ids+=[r.id for r in removed_rxns]
                        m.remove_reactions(removed_rxns)
            #Remove condition-specific reactions
            if condition_specific_remove is not None:
                for cond in condition_specific_remove.keys():
                    if condition==cond:
                        removed_rxns=[m.reactions.get_by_id(r) for r in condition_specific_remove[cond] if r in model.reactions]
                        removed_ids+=[r.id for r in removed_rxns]
                        m.remove_reactions(removed_rxns)
            print(f'Removed {removed_ids} due to carbon source or condition specific parameter.')
            reversibilities=[int(r.reversibility) for r in m.reactions]
            reactions_ids=[r.id for r in m.reactions]
            metabolite_ids=[m.id for m in m.metabolites]
            S=cobra.util.array.create_stoichiometric_matrix(m,array_type='DataFrame')
            S=S.values


            filename_mat=f'ich360red_{carbon_source}_{condition}_efmtool_data.mat'
            path_mat=os.path.join(out_path,filename_mat)
            
            savemat(path_mat,{'S':S,'reversibilities':reversibilities,'reaction_ids':reactions_ids,'metabolite_ids':metabolite_ids})
        



In [4]:
carbon_sources=['glc__D','lac__D','ac','pyr' ,'glyc','rib__D','xyl__D','succ','akg']
conditions=['aerobic','anaerobic']
carbon_source_specific_remove=None
condition_specific_remove={}

out={}
for substrate in tqdm.tqdm(carbon_sources):
    out[substrate]={}
    for condition in conditions:
        export_matlab_structure(model,
                       substrate,
                       condition,
                       carbon_source_specific_remove=carbon_source_specific_remove,
                       condition_specific_remove=condition_specific_remove,
                       )

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 100.95it/s]

Exporting EFMTools Matlab structure for glc__D aerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for glc__D anaerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for lac__D aerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for lac__D anaerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for ac aerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for ac anaerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for pyr aerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMTools Matlab structure for pyr anaerobic....
Removed [] due to carbon source or condition specific parameter.
Exporting EFMT